In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 11: Seating System ---</h2><p>Your plane lands with plenty of time to spare. The final leg of your journey is a ferry that goes directly to the tropical island where you can finally start your vacation. As you reach the waiting area to board the ferry, you realize you're so early, nobody else has even arrived yet!</p>
<p>By modeling the process people use to choose (or abandon) their seat in the waiting area, you're pretty sure you can predict the best place to sit. You make a quick map of the seat layout (your puzzle input).</p>
<p>The seat layout fits neatly on a grid. Each position is either floor (<code>.</code>), an empty seat (<code>L</code>), or an occupied seat (<code>#</code>). For example, the initial seat layout might look like this:</p>
<pre><code>L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
</code></pre>
<p>Now, you just need to model the people who will be arriving shortly. Fortunately, people are entirely predictable and always follow a simple set of rules. All decisions are based on the <em>number of occupied seats</em> adjacent to a given seat (one of the eight positions immediately up, down, left, right, or diagonal from the seat). The following rules are applied to every seat simultaneously:</p>
<ul>
<li>If a seat is <em>empty</em> (<code>L</code>) and there are <em>no</em> occupied seats adjacent to it, the seat becomes <em>occupied</em>.</li>
<li>If a seat is <em>occupied</em> (<code>#</code>) and <em>four or more</em> seats adjacent to it are also occupied, the seat becomes <em>empty</em>.</li>
<li>Otherwise, the seat's state does not change.</li>
</ul>
<p><span title="Floor... floor never changes.">Floor (<code>.</code>) never changes</span>; seats don't move, and nobody sits on the floor.</p>
<p>After one round of these rules, every seat in the example layout becomes occupied:</p>
<pre><code>#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
</code></pre>
<p>After a second round, the seats with four or more occupied adjacent seats become empty again:</p>
<pre><code>#.LL.L#.##
#LLLLLL.L#
L.L.L..L..
#LLL.LL.L#
#.LL.LL.LL
#.LLLL#.##
..L.L.....
#LLLLLLLL#
#.LLLLLL.L
#.#LLLL.##
</code></pre>
<p>This process continues for three more rounds:</p>
<pre><code>#.##.L#.##
#L###LL.L#
L.#.#..#..
#L##.##.L#
#.##.LL.LL
#.###L#.##
..#.#.....
#L######L#
#.LL###L.L
#.#L###.##
</code></pre>
<pre><code>#.#L.L#.##
#LLL#LL.L#
L.L.L..#..
#LLL.##.L#
#.LL.LL.LL
#.LL#L#.##
..L.L.....
#L#LLLL#L#
#.LLLLLL.L
#.#L#L#.##
</code></pre>
<pre><code>#.#L.L#.##
#LLL#LL.L#
L.#.L..#..
#L##.##.L#
#.#L.LL.LL
#.#L#L#.##
..L.L.....
#L#L##L#L#
#.LLLLLL.L
#.#L#L#.##
</code></pre>
<p>At this point, something interesting happens: the chaos stabilizes and further applications of these rules cause no seats to change state! Once people stop moving around, you count <em><code>37</code></em> occupied seats.</p>
<p>Simulate your seating area by applying the seating rules repeatedly until no seats change state. <em>How many seats end up occupied?</em></p>
</article>


In [2]:
from copy import deepcopy


example = """
L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""


class SeatLayout:
    neigbors = ((-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1))

    def __init__(self, s: str) -> None:
        self.layout = [list(l) for l in s.strip().splitlines()]
        self.rows, self.cols = len(self.layout), len(self.layout[0])

    def __repr__(self) -> str:
        return "\n".join("".join(l) for l in self.layout)

    def occupied_neigbors(self, row: int, col: int) -> int:
        return sum(
            1
            for dr, dc in self.neigbors
            if 0 <= row + dr < self.rows
            and 0 <= col + dc < self.cols
            and self.layout[row + dr][col + dc] == "#"
        )

    def round(self) -> SeatLayout:
        next_sl = deepcopy(self)

        for r, c in product(range(self.rows), range(self.cols)):
            if self.layout[r][c] == "L" and self.occupied_neigbors(r, c) == 0:
                next_sl.layout[r][c] = "#"
            elif self.layout[r][c] == "#" and self.occupied_neigbors(r, c) >= 4:
                next_sl.layout[r][c] = "L"
            else:
                next_sl.layout[r][c] = self.layout[r][c]

        return next_sl

    def occupied_seats(self) -> int:
        return sum(
            1
            for r, c in product(range(self.rows), range(self.cols))
            if self.layout[r][c] == "#"
        )

    @classmethod
    def stabilized_occupied_seats(cls, s: str) -> int:
        seen = set()
        current = cls(s)
        current_str = str(current)

        while current_str not in seen:
            seen.add(current_str)
            current = current.round()
            current_str = str(current)

        return current.occupied_seats()


assert SeatLayout.stabilized_occupied_seats(example) == 37

In [3]:
with open("../input/day11.txt") as f:
    puzzle = f.read()

print(f"Part I: {SeatLayout.stabilized_occupied_seats(puzzle)}")

Part I: 2204


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>2204</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>As soon as people start to arrive, you realize your mistake. People don't just care about adjacent seats - they care about <em>the first seat they can see</em> in each of those eight directions!</p>
<p>Now, instead of considering just the eight immediately adjacent seats, consider the <em>first seat</em> in each of those eight directions. For example, the empty seat below would see <em>eight</em> occupied seats:</p>
<pre><code>.......#.
...#.....
.#.......
.........
..#L....#
....#....
.........
#........
...#.....
</code></pre>
<p>The leftmost empty seat below would only see <em>one</em> empty seat, but cannot see any of the occupied ones:</p>
<pre><code>.............
.L.L.#.#.#.#.
.............
</code></pre>
<p>The empty seat below would see <em>no</em> occupied seats:</p>
<pre><code>.##.##.
#.#.#.#
##...##
...L...
##...##
#.#.#.#
.##.##.
</code></pre>
<p>Also, people seem to be more tolerant than you expected: it now takes <em>five or more</em> visible occupied seats for an occupied seat to become empty (rather than <em>four or more</em> from the previous rules). The other rules still apply: empty seats that see no occupied seats become occupied, seats matching no rule don't change, and floor never changes.</p>
<p>Given the same starting layout as above, these new rules cause the seating area to shift around as follows:</p>
<pre><code>L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
</code></pre>
<pre><code>#.##.##.##
#######.##
#.#.#..#..
####.##.##
#.##.##.##
#.#####.##
..#.#.....
##########
#.######.#
#.#####.##
</code></pre>
<pre><code>#.LL.LL.L#
#LLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLLL.L
#.LLLLL.L#
</code></pre>
<pre><code>#.L#.##.L#
#L#####.LL
L.#.#..#..
##L#.##.##
#.##.#L.##
#.#####.#L
..#.#.....
LLL####LL#
#.L#####.L
#.L####.L#
</code></pre>
<pre><code>#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##LL.LL.L#
L.LL.LL.L#
#.LLLLL.LL
..L.L.....
LLLLLLLLL#
#.LLLLL#.L
#.L#LL#.L#
</code></pre>
<pre><code>#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.#L.L#
#.L####.LL
..#.#.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
</code></pre>
<pre><code>#.L#.L#.L#
#LLLLLL.LL
L.L.L..#..
##L#.#L.L#
L.L#.LL.L#
#.LLLL#.LL
..#.L.....
LLL###LLL#
#.LLLLL#.L
#.L#LL#.L#
</code></pre>
<p>Again, at this point, people stop shifting around and the seating area reaches equilibrium. Once this occurs, you count <em><code>26</code></em> occupied seats.</p>
<p>Given the new visibility method and the rule change for occupied seats becoming empty, once equilibrium is reached, <em>how many seats end up occupied?</em></p>
</article>

</main>


In [4]:
class SeatLayoutII(SeatLayout):
    def occupied_neigbors(self, row: int, col: int) -> int:
        occupied = 0
        for dr, dc in self.neigbors:
            step = 1
            while (
                0 <= row + step * dr < self.rows
                and 0 <= col + step * dc < self.cols
                and self.layout[row + step * dr][col + step * dc] == "."
            ):
                step += 1

            if (
                0 <= row + step * dr < self.rows
                and 0 <= col + step * dc < self.cols
                and self.layout[row + step * dr][col + step * dc] == "#"
            ):
                occupied += 1
        return occupied

    def round(self) -> SeatLayout:
        next_sl = deepcopy(self)

        for r, c in product(range(self.rows), range(self.cols)):
            if self.layout[r][c] == "L" and self.occupied_neigbors(r, c) == 0:
                next_sl.layout[r][c] = "#"
            elif self.layout[r][c] == "#" and self.occupied_neigbors(r, c) >= 5:
                next_sl.layout[r][c] = "L"
            else:
                next_sl.layout[r][c] = self.layout[r][c]

        return next_sl


assert SeatLayoutII.stabilized_occupied_seats(example) == 26

In [5]:
print(f"Part II: {SeatLayoutII.stabilized_occupied_seats(puzzle)}")

Part II: 1986


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>1986</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
